# Проект по SQL. Анализ БД сервиса для чтения книг

**Цель исследования:**

Анализ базы данных крупного сервиса для чтения книг по подписке.

**Техническое задание:**

    1. Посчитайте, сколько книг вышло после 1 января 2000 года;
    2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
    3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
    4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
    5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

# План работы:

    1. Подключение библиотек и загрузка базы данных;
    2. Выполнение задания;
    3. Вывод

##  Подключение библиотек и загрузка базы данных

In [1]:
!pip install psycopg2-binary
import pandas as pd
import warnings; warnings.filterwarnings(action='once')

from sqlalchemy import create_engine

In [2]:
# устанавливаем параметры

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                     db_config['pwd'],
                     db_config['host'],
                     db_config['port'],
                     db_config['db']) 

# сохраняем коннектор

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Знакомство с таблицами

**Таблица books:**

In [3]:
query = '''

SELECT *
FROM books
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Таблица authors:**

In [4]:
query = '''

SELECT *
FROM authors
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица publishers:**

In [5]:
query = '''

SELECT *
FROM publishers
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица ratings:**

In [6]:
query = '''

SELECT *
FROM ratings
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица reviews:**

In [7]:
query = '''

SELECT *
FROM reviews
LIMIT 5

'''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Выполнение задания

### Посчитаем, сколько книг вышло после 1 января 2000 года;

In [8]:
query = '''
SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01'
'''

pd.io.sql.read_sql(query, con = engine)

,count
0,819


819 книг вышло после 1 января 2020 года.

### Для каждой книги посчитаем количество обзоров и среднюю оценку;

In [20]:
query = '''
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT rv.review_id),
       ROUND(AVG(rt.rating),2)
FROM books AS b
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id 
LEFT JOIN reviews AS rv ON b.book_id = rv.book_id 
GROUP BY b.book_id
ORDER BY count DESC, round DESC

'''

pd.io.sql.read_sql(query, con = engine)

,book_id,title,count,round
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


Всего в каталоге 1000 книги, максимальное количество обзоров 7. Средний рейтинг каждой книги представлен в таблице.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [10]:
query = '''
SELECT pb.publisher

FROM books AS b
JOIN publishers AS pb ON b.publisher_id = pb.publisher_id
WHERE num_pages > 50
GROUP BY pb.publisher
ORDER BY COUNT(b.book_id) DESC
LIMIT 1

'''

pd.io.sql.read_sql(query, con = engine)

,publisher
0,Penguin Books


Наиболее широко представлены книги издательства Penguin Books. При определении издательства не учитывались книги с толщиной менее 50 страниц.

### Определим автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [13]:
query = '''
WITH 

tb AS (
        SELECT book_id
        FROM ratings
        GROUP BY book_id
        HAVING COUNT(rating_id) > 50
        )

SELECT au.author_id, 
       au.author, 
       ROUND(AVG(rt.rating),2)
FROM books AS b
INNER JOIN tb ON b.book_id = tb.book_id
INNER JOIN authors AS au ON b.author_id = au.author_id
INNER JOIN ratings AS rt ON tb.book_id = rt.book_id

GROUP BY au.author_id
ORDER BY AVG(rt.rating) DESC

LIMIT 1

'''

pd.io.sql.read_sql(query, con = engine)

,author_id,author,round
0,236,J.K. Rowling/Mary GrandPré,4.29


Самый популярный автор со средним рейтингом **4,29** баллов из 5 – это тандем J.K. Rowling & Mary GrandPré

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [12]:
query = '''

SELECT COUNT(DISTINCT rv.text)/ COUNT(DISTINCT rv.username) AS AVG_rating

FROM ratings AS rt
INNER JOIN reviews AS rv ON rt.username = rv.username
WHERE rt.username in ( SELECT username
                      FROM ratings
                      GROUP BY username
                      HAVING COUNT(rating_id) > 50
                    )

'''
pd.io.sql.read_sql(query, con = engine)

,avg_rating
0,24


Среднее количество обзоров от пользователя составила 24 книги. В учет не брались пользователи оставившие менее 50 оценок

## Вывод

В ходе исследования была проанализирована база данных крупного сервиса чтения книг по подписке. В БД содержится информация о книгах, авторах, издательствах, а также пользовательские обзоры и оценки книг.
В ходе исследования было выявлено:
1.	Всего в каталоге 1000 книг, 819 из них были добавлены с 1 января 2000 года.
2.	Почти у каждой книги есть хотя бы один обзор от пользователей, максимально количество 7 обзоров на одну книгу.
3.	Наиболее широко представлены книги издательства Penguin Books. При определении издательства не учитывались книги с толщиной менее 50 страниц.
4.	Самый популярный автор со средним рейтингом 4,29 баллов из 5 – это тандем J.K. Rowling & Mary GrandPré
5.	Среднее количество обзоров от пользователя составила 24 книги. В учет не брались пользователи, оставившие менее 50 оценок.
